## Download the Data

In [4]:
import urllib.request
import zipfile
import os
import numpy as np

def download_datasets():
    # Download the folder
    !gdown --folder 1579owXfSSeM8FZq3DdoauCUofMCahdzV

    # Find all CSV files in the datasets folders
    csv_files = []
    for root, dirs, files in os.walk("datasets"):
        for file in files:
            if file.endswith(".csv"):
                csv_files.append(os.path.join(root, file))

    # Load the CSV files as numpy matrices
    datasets = {}
    for file in csv_files:
        matrix = np.loadtxt(file, delimiter=",")
        file_name = os.path.basename(file).split('.')[0]
        datasets[file_name] = matrix

    return datasets

In [5]:
datasets = download_datasets()

Retrieving folder list
Processing file 1uInvZ0oug5wAOcBHWc1WT3JMUFWsM95H .DS_Store
Retrieving folder 1pujZYM8Lc9oEZhBvyv_U_WJ7YB25hjWA concrete
Processing file 19zsGCG9lulysdOeYhljjfZAZ8hpyvdU1 concrete-test.csv
Processing file 1COSlcnzXPzYBJOMnQhNknG0w02SkJs5Q concrete-train.csv
Retrieving folder 146_OyoT-zTjaoD-usJ-vA6N1OxT91i2n synth1
Processing file 1PM1_BjetTjhEFnnrxWVzgo11T8SXc9-7 synth1-test.csv
Processing file 1mFIU2jNTQYm7QOJRKFmB7iYGxCzf_Bpb synth1-train.csv
Retrieving folder 1tk_P3AVPamlx-EPgSHNylezwiY42BDTj synth2
Processing file 1qvN1jK21Ls4Y1-BkfASlGDNOzGe7QEld synth2-test.csv
Processing file 1LXMTakd6DfcujHD3IVrrwOjHDHlk1JyT synth2-train.csv
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1uInvZ0oug5wAOcBHWc1WT3JMUFWsM95H
To: /content/datasets/.DS_Store
100% 6.15k/6.15k [00:00<00:00, 21.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=19zsGCG9lulysdOeYhljjfZ

# Decisões de Implementação

1. **Representação de um indivíduo (genótipo):**
##### Árvore de expressão, na qual os nós internos são operadores (como adição, subtração, multiplicação e divisão) e os nós folhas são terminais (variáveis e constantes).

2. **Geração da população inicial:**
##### A população inicial é gerada aleatoriamente, com árvores de expressão de diferentes tamanhos e formas. Define-se uma profundidade máxima permitida para as árvores. Para adicionar diversidade à população inicial, utiliza-se uma combinação de métodos de geração, como o método de crescimento completo e o método de crescimento rampa.

3. **Operadores genéticos:**
##### A reprodução envolve a cópia direta de um indivíduo selecionado para a próxima geração. O cruzamento consiste em escolher dois indivíduos e trocar uma subárvore entre eles, criando dois novos indivíduos. A mutação envolve a alteração aleatória de um nó em uma árvore, substituindo-o por outro nó compatível (um operador por outro operador, ou um terminal por outro terminal).

4. **Facilidades para variação de parâmetros:**
##### Em vez de usar parâmetros hardcoded, foi criado uma configuração centralizada, (uma estrutura de configuração), que pode ser facilmente alterada. Isso permitirá o ajuste dos parâmetros, como a taxa de cruzamento, a taxa de mutação, o tamanho da população, o número máximo de gerações e a profundidade máxima da árvore, sem modificar o código principal. Essa abordagem facilita a análise de sensibilidade dos parâmetros e a avaliação das soluções alcançadas.

# Tentativo 1 de Implementação


In [3]:
pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 34.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import random
import operator
import numpy as np

### **Node**

In [31]:
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

class Tree:
    def __init__(self, max_depth, terminals, operators):
        self.root = None
        self.max_depth = max_depth
        self.terminals = terminals
        self.operators = operators
    
    def generate_random_tree(self, depth):
        if depth >= self.max_depth:
            return Node(random.choice(self.terminals))
        
        node = Node(random.choice(self.operators))
        node.left = self.generate_random_tree(depth+1)
        node.right = self.generate_random_tree(depth+1)
        return node
        
    def evaluate(self, x_values):
        return self._evaluate_node(self.root, x_values)
    
    def _evaluate_node(self, node, x_values):
        if node is None:
            return 0.0
        
        if isinstance(node.value, str):
            if node.value in self.terminals:
                return x_values[self.terminals.index(node.value)]
            else:
                return 0.0
        
        left_val = self._evaluate_node(node.left, x_values)
        right_val = self._evaluate_node(node.right, x_values)
        
        if node.value == '+':
            return left_val + right_val
        elif node.value == '-':
            return left_val - right_val
        elif node.value == '*':
            return left_val * right_val
        elif node.value == '/':
            if right_val != 0.0:
                return left_val / right_val
            else:
                return 0.0
        
        return 0.0
    
    def infix_expression(self):
        return self._infix_expression_node(self.root)
    
    def _infix_expression_node(self, node):
        if isinstance(node.value, str):
            return node.value
        
        left_expr = self._infix_expression_node(node.left)
        right_expr = self._infix_expression_node(node.right)
        
        if node.value == '+':
            return '(' + left_expr + ' + ' + right_expr + ')'
        elif node.value == '-':
            return '(' + left_expr + ' - ' + right_expr + ')'
        elif node.value == '*':
            return '(' + left_expr + ' * ' + right_expr + ')'
        elif node.value == '/':
            return '(' + left_expr + ' / ' + right_expr + ')'
        
        return ''


In [49]:
t = Tree(4, ["x"], ["+", "-"])
t.root = t.generate_random_tree(4)

In [50]:
t.infix_expression()

'x'

In [ ]:
t

In [103]:
import random
import math

# define the non-terminal symbols
class Expr:
    def __init__(self, left, op, right):
        self.left = left
        self.op = op
        self.right = right
        
    def evaluate(self, variables):
        left_value = self.left.evaluate(variables)
        right_value = self.right.evaluate(variables)
        if self.op == "+":
            return left_value + right_value
        elif self.op == "-":
            return left_value - right_value
        elif self.op == "*":
            return left_value * right_value
        elif self.op == "/":
            return left_value / right_value
        else:
            raise ValueError(f"Unknown operator: {self.op}")

class Func:
    def __init__(self, name, arg):
        self.name = name
        self.arg = arg
        
    def evaluate(self, variables):
        arg_value = self.arg.evaluate(variables)
        if self.name == "sin":
            return math.sin(arg_value)
        elif self.name == "cos":
            return math.cos(arg_value)
        elif self.name == "exp":
            return math.exp(arg_value)
        elif self.name == "log":
            return math.log(arg_value)
        else:
            raise ValueError(f"Unknown function: {self.name}")
        
class Var:
    def __init__(self, name):
        self.name = name
        
    def evaluate(self, variables):
        return variables[self.name]
        
class Constant:
    def __init__(self, value):
        self.value = value
        
    def evaluate(self, variables):
        return self.value

# define the terminal symbols
operators = ["+", "-", "*", "/"]
functions = ["sin", "cos", "exp", "log"]
variables = ["x1", "x2"]
constants = ["a", "b", "c"]

CONSTANT_RATE = 0.5
FUNCTION_RATE = 0

# define the production rules
def generate_expr(depth):
    if depth <= 0:
        if random.random() < CONSTANT_RATE:
          return Constant(random.uniform(-10, 10))
        else:
          return Var("x")
    elif random.random() < FUNCTION_RATE:
        name = random.choice(functions)
        arg = generate_expr(depth-1)
        return Func(name, arg)
    else:
        left = generate_expr(depth-1)
        op = random.choice(operators)
        right = generate_expr(depth-1)

        return Expr(left, op, right)

def generate_expr_iter(depth, variables):
    stack = []
    for i in range(depth):
        if random.random() < FUNCTION_RATE:
            name = random.choice(functions)
            arg = stack.pop()
            stack.append(Func(name, arg))
        else:
            if random.random() < CONSTANT_RATE:
                stack.append(Constant(random.uniform(-10, 10)))
            else:
                stack.append(Var(random.choice(variables)))
            while len(stack) > 1:
                right = stack.pop()
                left = stack.pop()
                op = random.choice(operators)
                stack.append(Expr(left, op, right))
    return stack[0]


def generate_var():
    return Var(random.choice(variables))

def generate_constant():
    return Constant(random.uniform(-10, 10))

# define the start symbol
def generate_individual(depth):
    return generate_expr(depth)


NameError: ignored

In [62]:
def print_individual(individual):
    if isinstance(individual, Expr):
        left_str = print_individual(individual.left)
        right_str = print_individual(individual.right)
        return f"({left_str} {individual.op} {right_str})"
    elif isinstance(individual, Func):
        arg_str = print_individual(individual.arg)
        return f"{individual.name}({arg_str})"
    elif isinstance(individual, Var):
        return individual.name
    elif isinstance(individual, Constant):
        return str(individual.value)
    else:
        raise ValueError(f"Unknown individual type: {type(individual)}")


In [101]:
# test the grammar
individual = generate_individual(2)
individual

-
-
/


In [68]:
variables = {"x1": 1, "x2": 1, "x3": 1, "x4": 1, "x5": 1}
print(individual.evaluate(variables))

17.18067094813396


In [102]:
print_individual(individual)

'((x - x) - (x / x))'

In [52]:
X = [[1, 1, 2], [2, 2, 4]]

In [53]:
expr = Expr(None, "+", None)

In [54]:
expr.evaluate(X[0])

AttributeError: ignored

In [14]:
# Nó da árvore de expressão
class Node:
    def __init__(self, function=None, value=None):
        self.function = function
        self.value = value
        self.children = []

    def evaluate(self, x):
        if self.function:
            return self.function(*[child.evaluate(x) for child in self.children])
        else:
            return self.value(x)

In [21]:
expression = generate_random_expression(depth=3)
expression_string = print_expression(expression)
print(expression_string)

(((x1 + x2) * (x3 + x4)) / x5) = 0


In [25]:
expression.evaluate([1,1,1,1,2])

TypeError: ignored

### **Seleção**

In [ ]:
# Avaliação de fitness
def fitness(individual, X, Y):
    predictions = [individual.evaluate(x) for x in X]
    error = np.mean((Y - predictions)**2)
    return 1 / (1 + error)

def fitness(individual, X, Y):
    N = X.shape[0]
    errors = np.square(individual.evaluate(X) - Y)
    rmse = np.sqrt(np.sum(errors) / N)
    return rmse

# Seleção
def tournament_selection(population, X, Y, k=3):
    if k > len(population):
          k = len(population)

    selected = random.sample(population, k)
    fitnesses = [fitness(ind, X, Y) for ind in selected]
    return selected[np.argmax(fitnesses)]

### **Operadores Genéticos**

In [ ]:
# Cruzamento
def crossover(parent1, parent2):
    if not parent1.children or not parent2.children or random.random() > CROSSOVER_RATE:
        return parent1, parent2

    # Escolher ponto de cruzamento aleatoriamente em ambos os pais
    crossover_point1 = random.randint(0, len(parent1.children) - 1)
    crossover_point2 = random.randint(0, len(parent2.children) - 1)

    # Trocar subárvores entre os pais
    parent1.children[crossover_point1], parent2.children[crossover_point2] = parent2.children[crossover_point2], parent1.children[crossover_point1]

    return parent1, parent2

# Mutação
def mutate(individual):
    if not individual.children or random.random() > MUTATION_RATE:
        return individual

    mutation_point = random.randint(0, len(individual.children) - 1)
    individual.children[mutation_point] = generate_random_expression(MAX_DEPTH - 1)

    return individual

### **Geração da população inicial**

In [8]:
# Gerar população inicial
def generate_random_expression(depth):
    if depth == 0 or random.random() < 0.1:
        terminal = random.choice(TERMINALS)
        return Node(value=terminal)
    else:
        function = random.choice(list(FUNCTIONS.keys()))
        node = Node(function=FUNCTIONS[function])
        for _ in range(2):
            node.children.append(generate_random_expression(depth-1))
        return node

In [9]:
from graphviz import Digraph
from IPython.display import Image


def visualize_tree_graph(node, graph=None, parent=None):
    if graph is None:
        graph = Digraph('Expression Tree', format='png')
        
    if node.function:
        label = FUNCTION_MAP[node.function.__name__]
    else:
        label = TERMINAL_MAP[node.value.__name__]
        
    graph.node(str(id(node)), label)
    
    if parent:
        graph.edge(str(id(parent)), str(id(node)))

    for child in node.children:
        visualize_tree_graph(child, graph, node)

    return graph

In [5]:
root = generate_random_expression(3)
graph = visualize_tree_graph(root)
png_data = graph.pipe()
Image(png_data)

NameError: name 'generate_random_expression' is not defined

In [22]:
# Funções e terminais disponíveis
FUNCTION_MAP = {
    "add" : "+",
    "sub": "-",
    "mul": "*",
    "<lambda>": "/"
}
TERMINAL_MAP = {
    "<lambda>" : "x"
}


def print_expression(node):
  global x_count
  x_count = 0
  expr = print_expression_aux(node)
  return expr + " = y"

def print_expression_aux(node):
    global x_count
    if node.function:
        left_child = node.children[0]
        right_child = node.children[1]
        left_expression = print_expression_aux(left_child)
        right_expression = print_expression_aux(right_child)
        return f"({left_expression} {FUNCTION_MAP[node.function.__name__]} {right_expression})"
    else:
        x_count += 1
        return TERMINAL_MAP[node.value.__name__] + str(x_count)



In [16]:
# Example usage:
expression = generate_random_expression(depth=3)
expression_string = print_expression(expression)
print(expression_string)

(((x1 + x2) / (x3 + x4)) * ((x5 * x6) + (x7 * x8))) = 0


### **Algoritmo Genético**

In [ ]:
def genetic():
  population = [generate_random_expression(MAX_DEPTH) for _ in range(POPULATION_SIZE)]

  # Algoritmo genético
  X = np.random.uniform(-1, 1, size=(100, 1))
  Y = np.sin(X)

  for generation in range(NUM_GENERATIONS):
      new_population = []
      for _ in range(POPULATION_SIZE // 2):
          parent1 = tournament_selection(population, X, Y)
          parent2 = tournament_selection(population, X, Y)
          offspring1, offspring2 = crossover(parent1, parent2)
          offspring1 = mutate(offspring1)
          offspring2 = mutate(offspring2)
          new_population.extend([offspring1, offspring2])
          population = new_population

      # Avaliar e exibir o melhor indivíduo na geração atual
      best_individual = max(population, key=lambda ind: fitness(ind, X, Y))
      best_fitness = fitness(best_individual, X, Y)
      print(f"Melhor fitness na geração {generation + 1}: {best_fitness}")

  # Exibir o melhor indivíduo encontrado após todas as gerações
  best_individual = max(population, key=lambda ind: fitness(ind, X, Y))
  best_fitness = fitness(best_individual, X, Y)
  print(f"Melhor fitness geral: {best_fitness}")


### **Versão Debugável**

### **Funções e terminais**

In [12]:
# Funções e terminais disponíveis
FUNCTIONS = {
    "+": operator.add,
    "-": operator.sub,
    "*": operator.mul,
    "/": lambda a, b: a / b if b != 0 else 1
}
TERMINALS = [
    lambda x: x,
    lambda _: 1
]

### **Parâmetros**

In [ ]:
# Parâmetros
POPULATION_SIZE = 100
MAX_DEPTH = 5
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.9
NUM_GENERATIONS = 50

# **Testes**

In [ ]:
genetic()

Melhor fitness na geração 1: 0.6136256951327067
Melhor fitness na geração 2: 1.44759072825306e-13
Melhor fitness na geração 3: 0.7928148866866553
Melhor fitness na geração 4: 0.001352882253066952


KeyboardInterrupt: ignored

# Tentativo 2 de Implementação


In [1]:
POPSIZE: 1000
GENERATIONS: 50
ELITISM: 100                    # number of individuals that survive
PROB_CROSSOVER: 0.9
PROB_MUTATION: 0.05
TSIZE: 3
INCLUDE_GENOTYPE: False
SAVE_STEP: 500
VERBOSE: True
MIN_TREE_DEPTH: 10
MAX_TREE_DEPTH: 10
ADAPTIVE: False
LEARNING_FACTOR: 0.01

In [3]:
# Grammar
grammar = """<start> ::= <expr>
<expr> ::= <expr><op><expr>|(<expr><op><expr>)|<var>
<op> ::= +|-|*|\eb_div_\eb
<var> ::= x[0]|1.0
"""

In [13]:
grammar_dic = {
    "<start>" : ["<expr>"],
    "<expr>" : ["<expr><op><expr>","(<expr><op><expr>)","<var>"],
    "<op>" : ["+","-","*","/"],
    "<var>" : ["x", "1.0"]
}

start_rule = "<start>"

TERMINALS = grammar_dic["<op>"] + grammar_dic["<var>"]


In [ ]:
def generate_random_individual():
    genotype = [[] for _ in grammar_dic]
    tree_depth = grammar.recursive_individual_creation(genotype, start_rule, 0)
    return {'genotype': genotype, 'fitness': None, 'tree_depth' : tree_depth}

In [ ]:
max_init_depth = 5

In [ ]:
def recursive_individual_creation(genome, symbol, current_depth):
        codon = np.random.uniform()
        if current_depth > max_init_depth:
            prob_non_recursive = 0.0
            for rule in shortest_path[symbol][1:]:
                index = self.grammar[symbol].index(rule)
                prob_non_recursive += self.pcfg[self.index_of_non_terminal[symbol],index]
            prob_aux = 0.0
            for rule in self.shortest_path[(symbol,'NT')][1:]:
                index = self.grammar[symbol].index(rule)
                new_prob = self.pcfg[self.index_of_non_terminal[symbol],index] / prob_non_recursive
                prob_aux += new_prob
                if codon <= round(prob_aux,3):
                    expansion_possibility = index
                    break

In [14]:
for nt in grammar_dic.keys():
  

dict_keys(['<start>', '<expr>', '<op>', '<var>'])

In [17]:
import collections


class OrderedSet(collections.abc.MutableSet):
    """
    From http://code.activestate.com/recipes/528878-ordered-set/
    """

    def __init__(self, iterable=None):
        self.end = end = []
        end += [None, end, end]  # sentinel node for doubly linked list
        self.map = {}  # key --> [key, prev, next]
        if iterable is not None:
            self |= iterable

    def __len__(self):
        return len(self.map)

    def __contains__(self, key):
        return key in self.map

    def index(self, elem):
        if elem in self.map:
            return next(i for i, e in enumerate(self) if e == elem)
        else:
            raise KeyError("That element isn't in the set")

    def add(self, key):
        if key not in self.map:
            end = self.end
            curr = end[1]
            curr[2] = end[1] = self.map[key] = [key, curr, end]

    def discard(self, key):
        if key in self.map:
            key, prev, next = self.map.pop(key)
            prev[2] = next
            next[1] = prev

    def __iter__(self):
        end = self.end
        curr = end[2]
        while curr is not end:
            yield curr[0]
            curr = curr[2]

    def __reversed__(self):
        end = self.end
        curr = end[1]
        while curr is not end:
            yield curr[0]
            curr = curr[1]

    def pop(self, last=True):
        if not self:
            raise KeyError('set is empty')
        key = self.end[1][0] if last else self.end[2][0]
        self.discard(key)
        return key

    def __repr__(self):
        if not self:
            return '%s()' % (self.__class__.__name__,)
        return '%s(%r)' % (self.__class__.__name__, list(self))

    def __eq__(self, other):
        if isinstance(other, OrderedSet):
            return len(self) == len(other) and list(self) == list(other)
        return set(self) == set(other)

In [64]:
import re
import json
import numpy as np

class Grammar:
    """Class that represents a grammar. It works with the prefix notation."""
    NT = "NT"
    T = "T"
    NT_PATTERN = "(<.+?>)"
    RULE_SEPARATOR = "::="
    PRODUCTION_SEPARATOR = "|"

    def __init__(self, grammar_file=None):
        self.grammar_file = grammar_file
        self.grammar = {}
        self.productions_labels = {}
        self.non_terminals, self.terminals = set(), set()
        self.ordered_non_terminals = OrderedSet()
        self.non_recursive_options = {}
        self.number_of_options_by_non_terminal = None
        self.start_rule = None
        self.max_depth = None
        self.max_init_depth = None
        self.max_number_prod_rules = 0
        self.pcfg = None
        self.pcfg_mask = None
        self.pcfg_path = None
        self.index_of_non_terminal = {}
        self.shortest_path = {}

    def set_path(self, grammar_path):
        self.grammar_file = grammar_path

    def set_pcfg_path(self, pcfg_path):
        self.pcfg_path = pcfg_path

    def set_min_init_tree_depth(self, min_tree_depth):
        self.max_init_depth = min_tree_depth

    def set_max_tree_depth(self, max_tree_depth):
        self.max_depth = max_tree_depth

    def get_max_depth(self):
        return self.max_depth
    
    def get_max_init_depth(self):
        return self.max_init_depth

    def read_grammar(self):
        """
        Reads a Grammar in the BNF format and converts it to a python dictionary
        This method was adapted from PonyGE version 0.1.3 by Erik Hemberg and James McDermott
        """
        if self.grammar_file is None:
            raise Exception("You need to specify the path of the grammar file")


        with open(self.grammar_file, "r") as f:
            for line in f:
                if not line.startswith("#") and line.strip() != "":
                    if line.find(self.PRODUCTION_SEPARATOR):
                        left_side, productions = line.split(self.RULE_SEPARATOR)
                        left_side = left_side.strip()
                        if not re.search(self.NT_PATTERN, left_side):
                            raise ValueError("Left side not a non-terminal!")
                        self.non_terminals.add(left_side)
                        self.ordered_non_terminals.add(left_side)
                        # assumes that the first rule in the file is the axiom
                        if self.start_rule is None:
                            self.start_rule = (left_side, self.NT)
                        temp_productions = []
                        for production in [production.strip() for production in productions.split(self.PRODUCTION_SEPARATOR)]:
                            temp_production = []
                            if not re.search(self.NT_PATTERN, production):
                                if production == "None":
                                    production = ""
                                self.terminals.add(production)
                                temp_production.append((production, self.T))
                            else:
                                for value in re.findall("<.+?>|[^<>]*", production):
                                    if value != "":
                                        if re.search(self.NT_PATTERN, value) is None:
                                            sym = (value, self.T)
                                            self.terminals.add(value)
                                        else:
                                            sym = (value, self.NT)
                                        temp_production.append(sym)
                            temp_productions.append(temp_production)                          
                        self.max_number_prod_rules = max(self.max_number_prod_rules, len(temp_productions))
                        if left_side not in self.grammar:
                            self.grammar[left_side] = temp_productions
        
        if self.pcfg_path is not None:
            # load PCFG probabilities from json file. List of lists, n*n, with n = max number of production rules of a NT
            with open(self.pcfg_path) as f:
                self.pcfg = np.array(json.load(f))
        else:
            self.generate_uniform_pcfg()
        # self.compute_non_recursive_options()
        self.find_shortest_path()


    def find_shortest_path(self):
        open_symbols = []
        for nt in self.grammar.keys():
            depth = self.minimum_path_calc((nt,'NT'), open_symbols)
            
    def minimum_path_calc(self, current_symbol, open_symbols):
        if current_symbol[1] == self.T:
            return 0
        else:
            open_symbols.append(current_symbol)
            for derivation_option in self.grammar[current_symbol[0]]:
                max_depth = 0
                if current_symbol not in self.shortest_path:
                    self.shortest_path[current_symbol] = [999999]
                if bool(sum([i in open_symbols for i in derivation_option])):
                    continue
                if current_symbol not in derivation_option:
                    for symbol in derivation_option:
                        depth = self.minimum_path_calc(symbol, open_symbols)
                        depth += 1
                        if depth > max_depth:
                            max_depth = depth

                    if max_depth < self.shortest_path[current_symbol][0]:
                        self.shortest_path[current_symbol] = [max_depth]
                        if derivation_option not in self.shortest_path[current_symbol]:
                            self.shortest_path[current_symbol].append(derivation_option)
                    if max_depth == self.shortest_path[current_symbol][0]:
                        if derivation_option not in self.shortest_path[current_symbol]:
                            self.shortest_path[current_symbol].append(derivation_option)
            open_symbols.remove(current_symbol)
            return self.shortest_path[current_symbol][0]
                    
            

    def create_counter(self):
        self.counter = dict.fromkeys(self.grammar.keys(),[])
        for k in self.counter.keys():
            self.counter[k] = [0] * len(self.grammar[k])

    def generate_uniform_pcfg(self):
        """
        assigns uniform probabilities to grammar
        """
        array = np.zeros(shape=(len(self.grammar.keys()),self.max_number_prod_rules))
        for i, nt in enumerate(self.grammar):
            number_probs = len(self.grammar[nt])
            prob = 1.0 / number_probs
            array[i,:number_probs] = prob
            if nt not in self.index_of_non_terminal:
                self.index_of_non_terminal[nt] = i
        self.pcfg = array
        self.pcfg_mask = self.pcfg != 0

    def generate_random_pcfg(self):
        pass

    def get_mask(self):
        return self.pcfg_mask

    def get_index_of_non_terminal(self):
        return self.index_of_non_terminal

    def get_non_terminals(self):
        return self.ordered_non_terminals

    def count_number_of_options_in_production(self):
        if self.number_of_options_by_non_terminal is None:
            self.number_of_options_by_non_terminal = {}
            for nt in self.ordered_non_terminals:
                self.number_of_options_by_non_terminal.setdefault(nt, len(self.grammar[nt]))
        return self.number_of_options_by_non_terminal

    def list_non_recursive_productions(self, nt):
        non_recursive_elements = []
        for options in self.grammar[nt]:
            for option in options:
                if option[1] == self.NT and option[0] == nt:
                    break
            else:
                non_recursive_elements += [options]
        return non_recursive_elements

    def recursive_individual_creation(self, genome, symbol, current_depth):
        codon = np.random.uniform()
        if current_depth > self.max_init_depth:
            prob_non_recursive = 0.0
            for rule in self.shortest_path[(symbol,'NT')][1:]:
                index = self.grammar[symbol].index(rule)
                prob_non_recursive += self.pcfg[self.index_of_non_terminal[symbol],index]
            prob_aux = 0.0
            for rule in self.shortest_path[(symbol,'NT')][1:]:
                index = self.grammar[symbol].index(rule)
                new_prob = self.pcfg[self.index_of_non_terminal[symbol],index] / prob_non_recursive
                prob_aux += new_prob
                if codon <= round(prob_aux,3):
                    expansion_possibility = index
                    break
        else:
            prob_aux = 0.0
            for index, option in enumerate(self.grammar[symbol]):
                prob_aux += self.pcfg[self.index_of_non_terminal[symbol],index]
                if codon <= round(prob_aux,3):
                    expansion_possibility = index
                    break

        genome[self.get_non_terminals().index(symbol)].append([expansion_possibility,codon])
        expansion_symbols = self.grammar[symbol][expansion_possibility]
        depths = [current_depth]
        for sym in expansion_symbols:
            if sym[1] != self.T:
                depths.append(self.recursive_individual_creation(genome, sym[0], current_depth + 1))
        return max(depths)

    def mapping(self, mapping_rules, positions_to_map=None, needs_python_filter=False):
        if positions_to_map is None:
            positions_to_map = [0] * len(self.ordered_non_terminals)
        output = []
        max_depth = self._recursive_mapping(mapping_rules, positions_to_map, self.start_rule, 0, output)
        output = "".join(output)
        if self.grammar_file.endswith("pybnf"):
            output = self.python_filter(output)
        return output, max_depth

    def _recursive_mapping(self, mapping_rules, positions_to_map, current_sym, current_depth, output):
        depths = [current_depth]
        print(current_sym)
        print(self.T)
        if current_sym[1] == self.T:
            output.append(current_sym[0])
        else:
            current_sym_pos = self.ordered_non_terminals.index(current_sym[0])
            choices = self.grammar[current_sym[0]]
            codon = np.random.uniform()
            if positions_to_map[current_sym_pos] >= len(mapping_rules[current_sym_pos]):
                # Experiencia
                if current_depth > self.max_depth:
                    prob_non_recursive = 0.0
                    for rule in self.shortest_path[current_sym][1:]:
                        index = self.grammar[current_sym[0]].index(rule)
                        prob_non_recursive += self.pcfg[self.index_of_non_terminal[current_sym[0]],index]
                    prob_aux = 0.0
                    for rule in self.shortest_path[current_sym][1:]:
                        index = self.grammar[current_sym[0]].index(rule)
                        new_prob = self.pcfg[self.index_of_non_terminal[current_sym[0]],index] / prob_non_recursive
                        prob_aux += new_prob
                        if codon <= round(prob_aux,3):
                            expansion_possibility = index
                            break
                else:
                    prob_aux = 0.0
                    for index, option in enumerate(self.grammar[current_sym[0]]):
                        prob_aux += self.pcfg[self.index_of_non_terminal[current_sym[0]],index]
                        if codon <= round(prob_aux,3):
                            expansion_possibility = index
                            break
                mapping_rules[current_sym_pos].append([expansion_possibility,codon])
            else:
                # re-mapping with new probabilities
                print("Mapping Rules ", mapping_rules)
                print("Current Sym ", current_sym)
                print("Current Sym Pos ", current_sym_pos)
                print("Mapping Rules on Current Sym Pos ", mapping_rules[current_sym_pos])
                print("Positions to map ", positions_to_map)
                print("Positions to map on Current Sym Pos ", positions_to_map[current_sym_pos])
                print("codon without [1] ", mapping_rules[current_sym_pos][positions_to_map[current_sym_pos]])
                codon = mapping_rules[current_sym_pos][positions_to_map[current_sym_pos]]
                # If codon is list, get [1]
                if isinstance(codon, list):
                    codon = codon[1]
                if current_depth > self.max_depth:
                    prob_non_recursive = 0.0
                    for rule in self.shortest_path[(current_sym[0],'NT')][1:]:
                        index = self.grammar[current_sym[0]].index(rule)
                        prob_non_recursive += self.pcfg[self.index_of_non_terminal[current_sym[0]],index]
                    prob_aux = 0.0
                    for rule in self.shortest_path[(current_sym[0],'NT')][1:]:
                        index = self.grammar[current_sym[0]].index(rule)
                        new_prob = self.pcfg[self.index_of_non_terminal[current_sym[0]],index] / prob_non_recursive
                        prob_aux += new_prob
                        if codon <= round(prob_aux,3):
                            expansion_possibility = index
                            break
                else:
                    prob_aux = 0.0
                    for index, option in enumerate(self.grammar[current_sym[0]]):
                        prob_aux += self.pcfg[self.index_of_non_terminal[current_sym[0]],index]
                        if codon <= round(prob_aux,3):
                            expansion_possibility = index
                            break
            # update mapping rules com a updated expansion possibility
            mapping_rules[current_sym_pos][positions_to_map[current_sym_pos]] = [expansion_possibility, codon]
            current_production = expansion_possibility
            positions_to_map[current_sym_pos] += 1
            next_to_expand = choices[current_production]
            for next_sym in next_to_expand:
                depths.append(
                    self._recursive_mapping(mapping_rules, positions_to_map, next_sym, current_depth + 1, output))
        return max(depths)

    def compute_non_recursive_options(self):
        for key in self.grammar.keys():
            prob_non_recursive = 0.0
            non_recursive_prods = []
            for index, option in enumerate(self.grammar[key]):
                for s in option:
                    if s[0] == key:
                        break
                else:
                    prob_non_recursive += self.pcfg[self.index_of_non_terminal[key],index]
                    non_recursive_prods.append([index, option])
            self.non_recursive_options[key] = [non_recursive_prods, prob_non_recursive]

    def get_non_recursive_options(self, symbol):
        return self.non_recursive_options[symbol]


    def get_dict(self):
        return self.grammar

    def get_pcfg(self):
        return self.pcfg

    def get_shortest_path(self):
        return self.shortest_path

    @staticmethod
    def python_filter(txt):
        """ Create correct python syntax.
        We use {: and :} as special open and close brackets, because
        it's not possible to specify indentation correctly in a BNF
        grammar without this type of scheme."""
        txt = txt.replace("\le", "<=")
        txt = txt.replace("\ge", ">=")
        txt = txt.replace("\l", "<")
        txt = txt.replace("\g", ">")
        txt = txt.replace("\eb", "|")
        indent_level = 0
        tmp = txt[:]
        i = 0
        while i < len(tmp):
            tok = tmp[i:i+2]
            if tok == "{:":
                indent_level += 1
            elif tok == ":}":
                indent_level -= 1
            tabstr = "\n" + "  " * indent_level
            if tok == "{:" or tok == ":}" or tok == "\\n":
                tmp = tmp.replace(tok, tabstr, 1)
            i += 1
            # Strip superfluous blank lines.
            txt = "\n".join([line for line in tmp.split("\n") if line.strip() != ""])
        return txt

    def get_start_rule(self):
        return self.start_rule

    def __str__(self):
        grammar = self.grammar
        text = ""
        for key in self.ordered_non_terminals:
            text += key + " ::= "
            for options in grammar[key]:
                for option in options:
                    text += option[0]
                if options != grammar[key][-1]:
                    text += " | "
            text += "\n"
        return text

# Create one instance and export its methods as module-level functions.
# The functions share state across all uses
# (both in the user's code and in the Python libraries), but that's fine
# for most programs and is easier for the casual user


In [26]:
_inst = Grammar()
set_path = _inst.set_path
set_pcfg_path = _inst.set_pcfg_path
read_grammar = _inst.read_grammar
get_non_terminals = _inst.get_non_terminals
count_number_of_options_in_production = _inst.count_number_of_options_in_production
list_non_recursive_productions = _inst.list_non_recursive_productions
recursive_individual_creation = _inst.recursive_individual_creation
mapping = _inst.mapping
start_rule = _inst.get_start_rule
set_max_tree_depth = _inst.set_max_tree_depth
set_min_init_tree_depth = _inst.set_min_init_tree_depth
get_max_depth = _inst.get_max_depth
get_non_recursive_options = _inst.get_non_recursive_options
# compute_non_recursive_options = _inst.compute_non_recursive_options
get_dict = _inst.get_dict
get_pcfg = _inst.get_pcfg
get_mask = _inst.get_mask
get_shortest_path = _inst.get_shortest_path
get_index_of_non_terminal = _inst.get_index_of_non_terminal
ordered_non_terminals = _inst.ordered_non_terminals
max_init_depth = _inst.get_max_init_depth
python_filter = _inst.python_filter

In [65]:
g = Grammar("grammars/regression.txt")
g.read_grammar()

In [66]:
g.read_grammar()
g.set_max_tree_depth(9)

In [48]:
genome = [[0], [0, 3, 3], [0], [], [1, 1]]

In [49]:
mapping_numbers = [0] * len(genome)

In [68]:
g.grammar

{'<start>': [[('<expr>', 'NT')]],
 '<expr>': [[('<expr>', 'NT'), ('<op>', 'NT'), ('<expr>', 'NT')],
  [('(', 'T'), ('<expr>', 'NT'), ('<op>', 'NT'), ('<expr>', 'NT'), (')', 'T')],
  [('<var>', 'NT')]],
 '<op>': [[('+', 'T')], [('-', 'T')], [('*', 'T')], [('\\eb_div_\\eb', 'T')]],
 '<var>': [[('x[0]', 'T')], [('1.0', 'T')]]}

In [69]:
g.start_rule

('<start>', 'NT')

In [67]:
print(g.mapping(genome, mapping_numbers, needs_python_filter=True))

('<start>', 'NT')
T
('<expr>', 'NT')
T
Mapping Rules  [[[0, 0], [0, 0.8420644070046309], [0, 0.955463325123809]], [[0, 0], 3, 3], [0], [], [1, 1]]
Current Sym  ('<expr>', 'NT')
Current Sym Pos  1
Mapping Rules on Current Sym Pos  [[0, 0], 3, 3]
Positions to map  [3, 1, 0, 0, 0]
Positions to map on Current Sym Pos  1
codon without [1]  3


UnboundLocalError: local variable 'expansion_possibility' referenced before assignment

In [70]:
grammar_dic = {
            '<start>': [
                [('<expr>', 'NT')]
            ], 
            '<expr>': [
                [('<expr>', 'NT'), ('<op>', 'NT'), ('<expr>', 'NT')], 
                [('(', 'T'), ('<expr>', 'NT'), ('<op>', 'NT'), ('<expr>', 'NT'), (')', 'T')], 
                [('<var>', 'NT')]
            ],
            '<op>': [
                [('+', 'T')], 
                [('-', 'T')], 
                [('*', 'T')], 
                [('\\eb_div_\\eb', 'T')]
            ],
            '<var>': [
                [('x[0]', 'T')], 
                [('1.0', 'T')]
            ]
        }

In [71]:
shortest_path = {}

def minimum_path_calc(current_symbol, open_symbols):
        if current_symbol[1] == "T":
            return 0
        else:
            open_symbols.append(current_symbol)
            for derivation_option in grammar_dic[current_symbol[0]]:
                max_depth = 0
                if current_symbol not in shortest_path:
                    shortest_path[current_symbol] = [999999]
                if bool(sum([i in open_symbols for i in derivation_option])):
                    continue
                if current_symbol not in derivation_option:
                    for symbol in derivation_option:
                        depth = minimum_path_calc(symbol, open_symbols)
                        depth += 1
                        if depth > max_depth:
                            max_depth = depth

                    if max_depth < shortest_path[current_symbol][0]:
                        shortest_path[current_symbol] = [max_depth]
                        if derivation_option not in shortest_path[current_symbol]:
                            shortest_path[current_symbol].append(derivation_option)
                    if max_depth == shortest_path[current_symbol][0]:
                        if derivation_option not in shortest_path[current_symbol]:
                            shortest_path[current_symbol].append(derivation_option)
            open_symbols.remove(current_symbol)
            return shortest_path[current_symbol][0]

In [76]:
open_symbols = []
for nt in grammar_dic.keys():
    depth = minimum_path_calc((nt,'NT'), open_symbols)

In [77]:
shortest_path

{('<start>', 'NT'): [3, [('<expr>', 'NT')]],
 ('<expr>', 'NT'): [2, [('<var>', 'NT')]],
 ('<var>', 'NT'): [1, [('x[0]', 'T')], [('1.0', 'T')]],
 ('<op>', 'NT'): [1,
  [('+', 'T')],
  [('-', 'T')],
  [('*', 'T')],
  [('\\eb_div_\\eb', 'T')]]}

In [78]:
shortest_path2 = {
            ('<start>', 'NT'): [3, grammar_dic['<start>'][0]],
            ('<expr>', 'NT'): [2, grammar_dic['<expr>'][2]],
            ('<var>', 'NT'): [1, grammar_dic['<var>'][0], grammar_dic['<var>'][1]],
            ('<op>', 'NT'): [1,grammar_dic['<op>'][0], grammar_dic['<op>'][1],grammar_dic['<op>'][2], grammar_dic['<op>'][3]]
        }